In [1]:
!pip install transformers stellargraph nodevectors

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.4/2.4 MB 7.2 MB/s eta 0:00:00


     ---------------------------------------- 23.2/23.2 MB 7.7 MB/s eta 0:00:00
  Created wheel for nodevectors: filename=nodevectors-0.1.23-py3-none-any.whl size=17952 sha256=c5b1266e6b0feec63826b599842d12603a856e5e829ad53051f4f1fee880eab7
  Stored in directory: c:\users\marco\appdata\local\pip\cache\wheels\6e\6a\11\ac8620f101db6248c3cd436636eb17f6f7430cd1ac3a3068dc
  Created wheel for csrgraph: filename=csrgraph-0.1.28-py3-none-any.whl size=17636 sha256=7cb4780a10b1d06aada47ff5e8d1aafd0ea64b906f5e26811cee0b8b9f90d168
  Stored in directory: c:\users\marco\appdata\local\pip\cache\wheels\3d\0a\58\36a63805f1cd719cbcc5a3c4c109604985a809e7d81bac1649
Successfully built nodevectors csrgraph


In [2]:
#from transformers import AutoTokenizer
from transformers import BertTokenizer
from transformers import RobertaTokenizerFast
import matplotlib.pyplot as plt
from math import isclose
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
from stellargraph import StellarGraph, datasets
from stellargraph.data import EdgeSplitter
from collections import Counter
import multiprocessing
from sklearn.model_selection import train_test_split
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
from tqdm.notebook import tqdm

In [4]:
abstracts = dict()
with open('abstracts.txt', 'r', encoding='utf8') as f:
    for line in f:
        node, abstract = line.split('|--|')
        abstracts[int(node)] = abstract

In [5]:
from transformers import pipeline

vocab = {}
#cnt = 0
#summarizer = pipeline("summarization", device = 0)

for (idx, abst) in tqdm(abstracts.items()):
    # Initialize the HuggingFace summarization pipeline
    #summarized = summarizer(abst, min_length=75, max_length=100)
    #vocab.update(set(summarized[0]['summary_text'].split()))
    #vocab.update(set(abst.split()))
    '''
    for word in abst.split():
        if word not in vocab:
            vocab[word] = cnt
            cnt += 1
    '''
    for word in abst.split():
        if word in vocab:
          vocab[word] += 1
        else:
          vocab[word] = 1
    
print(len(vocab))
vocab = {word : i for (i, word) in enumerate(sorted(vocab, key=vocab.get)[:1500])}

  0%|          | 0/138499 [00:00<?, ?it/s]

381107


In [6]:
onehot_abstracts = {}
for (idx,abst) in tqdm(abstracts.items()):
    encoding = np.zeros(len(vocab))
    for word in abst.split():
        if word in vocab:
            encoding[vocab[word]] = 1
    onehot_abstracts[idx] = encoding

  0%|          | 0/138499 [00:00<?, ?it/s]

In [7]:
G = nx.read_edgelist('edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

Number of nodes: 138499
Number of edges: 1091955


In [5]:
from transformers import DistilBertTokenizerFast, DistilBertModel

#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [8]:
def tokenize_function(abstract):
    return tokenizer(abstract, truncation=True, return_tensors="pt")

tokenized_datasets = {}
for idx, a in tqdm(abstracts.items()):
    tokenized_datasets[idx] = tokenize_function(a)

  0%|          | 0/138499 [00:00<?, ?it/s]

In [9]:
from transformers import BertModel, BertConfig
from transformers import RobertaModel, RobertaConfig
from transformers import DistilBertModel, DistilBertConfig
import torch

#configuration = BertConfig()
#model = BertModel(configuration)

#configuration = RobertaConfig()
#model = RobertaModel(configuration)

#configuration = DistilBertConfig()
#model = DistilBertModel(configuration)

model = DistilBertModel.from_pretrained("distilbert-base-uncased")

device = torch.device('cuda')
model = model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
tokenized_datasets[0]

{'input_ids': tensor([[  101,  1996,  2458,  1997,  2019, 12978,  2291,  2005,  1996,  3737,
          7667,  1997, 23843,  2598,  7497,  1006, 12943,  2140,  1007,  1010,
          1999, 10388,  2007,  3378,  4781,  1998, 11433,  1010,  2003,  3591,
          1012,  1996,  2291,  2003,  3605,  1997,  2019,  3746, 13617,  1010,
          2872,  2503,  1996, 13828,  1997,  2019,  2948,  2000,  2501,  4871,
          1997,  1996, 12943,  2140,  2076,  1037,  3671,  6934,  2000,  2019,
         23843,  1012,  1037,  2944,  1011,  2241, 16134,  2003,  2109,  2000,
          2004, 17119, 18249,  1996, 23569, 28591,  2674,  2090,  1037, 23561,
          1997,  1996, 12943,  2140,  1998,  1996,  5025,  3746,  2951,  1999,
          2344,  2000, 18422,  1996,  2597,  1998, 10296,  1997,  1996,  4950,
          2012,  1996,  7107,  1996,  3746,  2001,  3734,  1012,  1996,  4950,
          2597,  1998, 10296,  2951,  2024,  2109,  2247,  2007,  1996, 22138,
          4462,  2504,  2005,  2169,  

In [19]:
out = model(**tokenized_datasets[1].to(device))
print(tokenized_datasets[1]['input_ids'].shape)
print(out.last_hidden_state.cpu().detach().numpy().shape)

torch.Size([1, 131])
(1, 131, 768)


In [20]:
g_feature_attr = G.copy()

In [8]:
def compute_features(node_id):
    #return [G.degree(node_id), node_id]
    #return [model(**tokenized_datasets[node_id].to(device)).last_hidden_state.cpu().detach().numpy()]
    return onehot_abstracts[node_id]

for node_id, node_data in tqdm(G.nodes(data=True)):
    node_data["feature"] = compute_features(node_id)

  0%|          | 0/138499 [00:00<?, ?it/s]

In [9]:
G.nodes[0], G.nodes[1]

({'feature': array([1., 1., 1., ..., 0., 0., 0.])},
 {'feature': array([0., 0., 0., ..., 0., 0., 0.])})

In [10]:
G = StellarGraph.from_networkx(G, node_features="feature")

MemoryError: Unable to allocate 792. MiB for an array with shape (1500, 138499) and data type float32

In [ ]:
print(G.info())

In [ ]:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, examples_test, labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global"
)

print(graph_test.info())

In [ ]:
edge_splitter_train = EdgeSplitter(graph_test, G)
graph_train, examples, labels = edge_splitter_train.train_test_split(
    p=0.1, method="global"
)
(
    examples_train,
    examples_model_selection,
    labels_train,
    labels_model_selection,
) = train_test_split(examples, labels, train_size=0.75, test_size=0.25)

print(graph_train.info())

In [ ]:
examples_train[0], labels_train[0]

In [ ]:
p = 1.0
q = 1.0
dimensions = 128
num_walks = 3
walk_length = 10
window_size = 10
num_iter = 1
workers = multiprocessing.cpu_count()

In [ ]:
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
import nodevectors
import csrgraph as cg

def node2vec_embedding(graph, name):
    '''
    rw = BiasedRandomWalk(graph)
    walks = rw.run(graph.nodes(), n=num_walks, length=walk_length, p=p, q=q)
    print(f"Number of random walks for '{name}': {len(walks)}")

    return walks

    
    model = Word2Vec(
        walks,
        size=dimensions,
        window=window_size,
        min_count=0,
        sg=1,
        workers=workers,
        iter=num_iter,
    )

    return model, walks

    def get_embedding(u):
        return model.wv[u]

    return get_embedding
    '''
    ggvec_model = nodevectors.GGVec() 
    embeddings = ggvec_model.fit_transform(cg.csrgraph(graph.to_networkx()))
    print(len(embeddings))
    def get_embedding(u):
      return embeddings[u]
    return get_embedding

In [ ]:
#embedding_train = node2vec_embedding(graph_train, "Train Graph")
embedding_train = node2vec_embedding(graph_train, "Train Graph")

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler


# 1. link embeddings
def link_examples_to_features(link_examples, transform_node, binary_operator):
    return [
        binary_operator(transform_node(src), transform_node(dst))
        for src, dst in link_examples
    ]


# 2. training classifier
def train_link_prediction_model(
    link_examples, link_labels, get_embedding, binary_operator
):
    clf = link_prediction_classifier()
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    clf.fit(link_features, link_labels)
    return clf


def link_prediction_classifier(max_iter=2000):
    lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring="roc_auc", max_iter=max_iter)
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])


# 3. and 4. evaluate classifier
def evaluate_link_prediction_model(
    clf, link_examples_test, link_labels_test, get_embedding, binary_operator
):
    link_features_test = link_examples_to_features(
        link_examples_test, get_embedding, binary_operator
    )
    score = evaluate_roc_auc(clf, link_features_test, link_labels_test)
    return score


def evaluate_roc_auc(clf, link_features, link_labels):
    predicted = clf.predict_proba(link_features)

    # check which class corresponds to positive links
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(link_labels, predicted[:, positive_column])

In [ ]:
def operator_hadamard(u, v):
    return u * v


def operator_l1(u, v):
    return np.abs(u - v)


def operator_l2(u, v):
    return (u - v) ** 2


def operator_avg(u, v):
    return (u + v) / 2.0


def run_link_prediction(binary_operator):
    clf = train_link_prediction_model(
        examples_train, labels_train, embedding_train, binary_operator
    )
    score = evaluate_link_prediction_model(
        clf,
        examples_model_selection,
        labels_model_selection,
        embedding_train,
        binary_operator,
    )

    return {
        "classifier": clf,
        "binary_operator": binary_operator,
        "score": score,
    }


binary_operators = [operator_hadamard, operator_l1, operator_l2, operator_avg]

In [ ]:
results = [run_link_prediction(op) for op in binary_operators]
best_result = max(results, key=lambda result: result["score"])

print(f"Best result from '{best_result['binary_operator'].__name__}'")

pd.DataFrame(
    [(result["binary_operator"].__name__, result["score"]) for result in results],
    columns=("name", "ROC AUC score"),
).set_index("name")

In [ ]:
embedding_test = node2vec_embedding(graph_test, "Test Graph")

In [ ]:
test_score = evaluate_link_prediction_model(
    best_result["classifier"],
    examples_test,
    labels_test,
    embedding_test,
    best_result["binary_operator"],
)
print(
    f"ROC AUC score on test set using '{best_result['binary_operator'].__name__}': {test_score}"
)

In [ ]:
# Calculate edge features for test data
link_features = link_examples_to_features(
    examples_test, embedding_test, best_result["binary_operator"]
)

# Learn a projection from 128 dimensions to 2
pca = PCA(n_components=2)
X_transformed = pca.fit_transform(link_features)

# plot the 2-dimensional points
plt.figure(figsize=(16, 12))
plt.scatter(
    X_transformed[:, 0],
    X_transformed[:, 1],
    c=np.where(labels_test == 1, "b", "r"),
    alpha=0.5,
)

In [ ]:
def predict_examples(
    clf, link_examples_test, get_embedding, binary_operator
):
    link_features_test = link_examples_to_features(
        link_examples_test, get_embedding, binary_operator
    )
    preds = evaluate(clf, link_features_test)
    return preds


def evaluate(clf, link_features):
    predicted = clf.predict_proba(link_features)
    return predicted

In [ ]:
def read_edgelist(filename, separator = ','):
    out = []
    with open(filename, 'r') as f:
        for line in f:
          line = line.split(separator)
          out.append([int(idx) for idx in line])
    return np.array(out)

In [ ]:
kaggle_examples = read_edgelist('/content/drive/MyDrive/data_challenge_2021/test.txt')

In [ ]:
preds = predict_examples(
    best_result["classifier"],
    kaggle_examples,
    embedding_test,
    best_result["binary_operator"],
)

In [ ]:
preds = preds[:, 1]

In [ ]:
import csv

predictions = zip(range(len(preds)), preds)
with open("submission.csv","w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for row in predictions:
        csv_out.writerow(row)